In [4]:
%jars ../explainability-core-experiments/counterfactuals/target/explainability-counterfactuals-jar-with-dependencies.jar

# Counterfactuals with OptaPlanner

In [5]:
Random random = new Random();
TimeUnit predictionTimeUnit = TimeUnit.SECONDS;
long predictionTimeOut = 10L;

In [6]:
import org.kie.kogito.explainability.model.*;
import org.kie.kogito.explainability.local.counterfactual.*;
import org.kie.kogito.explainability.local.counterfactual.TestUtils;
import org.kie.kogito.explainability.Config;
import org.kie.kogito.explainability.local.counterfactual.entities.CounterfactualEntity;

List<Output> goal = List.of(new Output("inside", Type.BOOLEAN, new Value<>(true), 1d));

List<Feature> features = new LinkedList<>();

List<FeatureBoundary> featureBoundaries = new LinkedList<>();
List<Boolean> constraints = new LinkedList<>();
features.add(FeatureFactory.newNumericalFeature("f-num1", 100.0));
constraints.add(false);
featureBoundaries.add(new FeatureBoundary(0.0, 1000.0));
features.add(FeatureFactory.newNumericalFeature("f-num2", 150.0));
constraints.add(false);
featureBoundaries.add(new FeatureBoundary(0.0, 1000.0));
features.add(FeatureFactory.newNumericalFeature("f-num3", 1.0));
constraints.add(false);
featureBoundaries.add(new FeatureBoundary(0.0, 1000.0));
features.add(FeatureFactory.newNumericalFeature("f-num4", 2.0));
constraints.add(false);
featureBoundaries.add(new FeatureBoundary(0.0, 1000.0));

final DataBoundaries dataBoundaries = new DataBoundaries(featureBoundaries);
CounterfactualExplainer counterfactualExplainer = new CounterfactualExplainer(dataBoundaries, constraints, goal, 5L, 70, 5000);

final double center = 500.0;
final double epsilon = 10.0;
PredictionInput input = new PredictionInput(features);
PredictionProvider model = TestUtils.getSumThresholdModel(center, epsilon);
PredictionOutput output = model.predictAsync(List.of(input))
        .get(Config.INSTANCE.getAsyncTimeout(), Config.INSTANCE.getAsyncTimeUnit())
        .get(0);
Prediction prediction = new Prediction(input, output);
List<CounterfactualEntity> counterfactualEntities = counterfactualExplainer.explainAsync(prediction, model)
        .get(predictionTimeOut, predictionTimeUnit);

double totalSum = 0;
for (CounterfactualEntity entity : counterfactualEntities) {
    totalSum += entity.asFeature().getValue().asNumber();
    System.out.println(entity);
}

[pool-2-thread-1] INFO org.optaplanner.core.impl.solver.DefaultSolver - Solving started: time spent (1), best score ([-1.0/0]hard/[0.0]soft), environment mode (REPRODUCIBLE), random (JDK with seed 0).
[pool-2-thread-1] INFO org.optaplanner.core.impl.localsearch.DefaultLocalSearchPhase - Local Search phase (0) ended: time spent (5000), best score ([0.0/0]hard/[-237.03978194829008]soft), score calculation speed (53351/sec), step total (6).
[pool-2-thread-1] INFO org.optaplanner.core.impl.solver.DefaultSolver - Solving ended: time spent (5000), best score ([0.0/0]hard/[-237.03978194829008]soft), score calculation speed (53319/sec), phase total (1), environment mode (REPRODUCIBLE).


DoubleFeature{value=100.09998642052497, doubleRangeMinimum=0.0, doubleRangeMaximum=1000.0, id='f-num1'}
DoubleFeature{value=150.0, doubleRangeMinimum=0.0, doubleRangeMaximum=1000.0, id='f-num2'}
DoubleFeature{value=237.9397955277651, doubleRangeMinimum=0.0, doubleRangeMaximum=1000.0, id='f-num3'}
DoubleFeature{value=2.0, doubleRangeMinimum=0.0, doubleRangeMaximum=1000.0, id='f-num4'}
